In [19]:
try:
    %tensorflow_version 2.x
except:
    pass

In [20]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np

In [21]:
#DATASET SOURCE FROM https://github.com/dbrehmer/Knowself/blob/master/data/mypersonality/essays.csv"
#DATASET USED IN THIS NOTEBOOK IS DATASET FROM THE SOURCE THAT HAS BEEN TRANSLATED TO BAHASA INDONESIA USING GOOGLE TRANSLATE

DATASET_URL = "https://raw.githubusercontent.com/lazuardi100/Hexa-Engineer/ML/Dataset/dataset.csv"
df = pd.read_csv(DATASET_URL, sep =',')

df.head()

,#AUTHID,TTEXT,cEXT,cNEU,cAGR,cCON,cOPN
0,1997_504851.txt,"Nah, sekarang saya baru saja bangun dari tidur...",n,y,y,n,y
1,1997_605191.txt,"Nah, di sini kita pergi dengan arus kesadaran ...",n,n,y,n,n
2,1997_687252.txt,Keyboard terbuka dan tombol untuk mendorong. H...,n,y,n,y,y
3,1997_568848.txt,Aku tidak percaya itu! Ini benar-benar terjadi...,y,n,y,y,n
4,1997_688160.txt,"Nah, di sini aku pergi dengan aliran tua yang ...",y,n,y,n,y


In [22]:
def changeLabel(labels):
  for index, values in enumerate(labels.values):
    if values == 'n':
      labels[index] = 0
    else:
      labels[index] = 1
  return labels

changeLabel(df['cEXT'])
changeLabel(df['cNEU'])
changeLabel(df['cAGR'])
changeLabel(df['cCON'])
changeLabel(df['cOPN'])

0       1
1       0
2       1
3       0
4       1
       ..
2462    0
2463    1
2464    0
2465    1
2466    1
Name: cOPN, Length: 2467, dtype: object

In [23]:
train_dataset, val_dataset, test_dataset = np.split(df, [int(.8 * len(df)), int(.9 * len(df))])


In [24]:
train_data = train_dataset['TTEXT']
train_label_ext  = train_dataset['cEXT']
train_label_neu  = train_dataset['cNEU']
train_label_agr  = train_dataset['cAGR']
train_label_con  = train_dataset['cCON']
train_label_opn  = train_dataset['cOPN']

val_data = val_dataset['TTEXT']
val_label_ext  = val_dataset['cEXT']
val_label_neu  = val_dataset['cNEU']
val_label_agr  = val_dataset['cAGR']
val_label_con  = val_dataset['cCON']
val_label_opn  = val_dataset['cOPN']

test_data = test_dataset['TTEXT']
test_label_ext  = test_dataset['cEXT']
test_label_neu  = test_dataset['cNEU']
test_label_agr  = test_dataset['cAGR']
test_label_con  = test_dataset['cCON']
test_label_opn  = test_dataset['cOPN']

In [25]:
train_dataset_ext = tf.data.Dataset.from_tensor_slices((train_data.values, train_label_ext.values.astype(dtype=np.float32)))
train_dataset_neu = tf.data.Dataset.from_tensor_slices((train_data.values, train_label_neu.values.astype(dtype=np.float32)))
train_dataset_agr = tf.data.Dataset.from_tensor_slices((train_data.values, train_label_agr.values.astype(dtype=np.float32)))
train_dataset_con = tf.data.Dataset.from_tensor_slices((train_data.values, train_label_con.values.astype(dtype=np.float32)))
train_dataset_opn = tf.data.Dataset.from_tensor_slices((train_data.values, train_label_opn.values.astype(dtype=np.float32)))

val_dataset_ext = tf.data.Dataset.from_tensor_slices((val_data.values, val_label_ext.values.astype(dtype=np.float32)))
val_dataset_neu = tf.data.Dataset.from_tensor_slices((val_data.values, val_label_neu.values.astype(dtype=np.float32)))
val_dataset_agr = tf.data.Dataset.from_tensor_slices((val_data.values, val_label_agr.values.astype(dtype=np.float32)))
val_dataset_con = tf.data.Dataset.from_tensor_slices((val_data.values, val_label_con.values.astype(dtype=np.float32)))
val_dataset_opn = tf.data.Dataset.from_tensor_slices((val_data.values, val_label_opn.values.astype(dtype=np.float32)))

test_dataset_ext = tf.data.Dataset.from_tensor_slices((test_data.values, test_label_ext.values.astype(dtype=np.float32)))
test_dataset_neu = tf.data.Dataset.from_tensor_slices((test_data.values, test_label_neu.values.astype(dtype=np.float32)))
test_dataset_agr = tf.data.Dataset.from_tensor_slices((test_data.values, test_label_agr.values.astype(dtype=np.float32)))
test_dataset_con = tf.data.Dataset.from_tensor_slices((test_data.values, test_label_con.values.astype(dtype=np.float32)))
test_dataset_opn = tf.data.Dataset.from_tensor_slices((test_data.values, test_label_opn.values.astype(dtype=np.float32)))

In [26]:
embedding = "https://tfhub.dev/google/nnlm-id-dim128/2"

hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)

model_ext = tf.keras.Sequential([
        hub_layer,
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')])

In [30]:
batch_size = 512
num_examples = tf.data.experimental.cardinality(train_dataset_ext).numpy()
ext_batches = train_dataset_ext.shuffle(num_examples // 4).batch(batch_size).prefetch(1)
val_batches = val_dataset_ext.batch(batch_size).prefetch(1)
test_batches = test_dataset_ext.batch(batch_size)

In [31]:
model_ext.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model_ext.fit(ext_batches,
                        validation_data=val_batches,
                        epochs=20)

Epoch 1/20
4/4 [==============================] - 5s 1s/step - loss: 0.4733 - accuracy: 0.8751 - val_loss: 0.6997 - val_accuracy: 0.5425
Epoch 2/20
4/4 [==============================] - 5s 1s/step - loss: 0.4567 - accuracy: 0.8932 - val_loss: 0.6316 - val_accuracy: 0.6518
Epoch 3/20
4/4 [==============================] - 5s 1s/step - loss: 0.4326 - accuracy: 0.8858 - val_loss: 0.6432 - val_accuracy: 0.6437
Epoch 4/20
4/4 [==============================] - 5s 1s/step - loss: 0.4061 - accuracy: 0.9407 - val_loss: 0.6572 - val_accuracy: 0.6194
Epoch 5/20
4/4 [==============================] - 5s 1s/step - loss: 0.3855 - accuracy: 0.9545 - val_loss: 0.6305 - val_accuracy: 0.6478
Epoch 6/20
4/4 [==============================] - 5s 1s/step - loss: 0.3645 - accuracy: 0.9513 - val_loss: 0.6526 - val_accuracy: 0.6316
Epoch 7/20
4/4 [==============================] - 5s 1s/step - loss: 0.3440 - accuracy: 0.9662 - val_loss: 0.6414 - val_accuracy: 0.6437
Epoch 8/20
4/4 [=========================